# Using Astroquery to find ALMA data of your science target

This tutorial walks you through using Astroquery's catalog and archive search functionality in addition to its line list services to find recent ALMA data on the CO 1-0 transition across M83. This can be adapted to similar extragalactic projects.

The Astroquery readthedocs is located [here](https://astroquery.readthedocs.io/en/latest/).

We suggest running Astroquery version 0.4.7. Some steps of this tutorial will not work in earlier versions, such as HSA search functionionality or ALMA authentication.

In [10]:
# Import Astroquery and check your current working version by doing the following:
import astroquery
print(astroquery.__version__)

# Import astroquery's ALMA archive search tool and other useful packages:
from astroquery.alma import Alma
import numpy as np
import astropy.units as u
#instantiate alma object/class
alma = Alma()

# If you have access to proprietary data that you want to work with, you will need to login to your ALMA account:
#login_name = ""
#alma.login(login_name)

0.4.6


## 1. Perform a query of the ALMA science archive

We perform an ALMA object query (`alma.query_object`). Astroquery sends the source name to the ALMA Archive, which then resolves the source coordinates using, i.e. Simbad, NED, and Vizier, for matches to the object name. Astroquery returns an astropy table containing information on all of the ALMA archival data towards M83's coordinates.

Note that you can instead perform a region search (`alma.query_region`) using Astroquery specifying a search radius. The default ALMA search radius is 10 arcminutes.

In [12]:
# Query the alma archive for galaxy M83 observations using an object query:
q_results = alma.query_object("M83")

# Astroquery returns an astropy table with all ALMA results matching the coordinates to M83.
print("Data for M83:")
print(q_results)

# Print the column names of the resulting table.
print("Column names of retrieved table:")
print(q_results.colnames)

Data for M83:
                           access_url                           access_format ... collections
                                                                              ...            
--------------------------------------------------------------- ------------- ... -----------
https://almascience.org/datalink/sync?ID=uid://A002/X3216af/X31     applicati ...            
https://almascience.org/datalink/sync?ID=uid://A002/X3216af/X31     applicati ...            
https://almascience.org/datalink/sync?ID=uid://A002/X3216af/X31     applicati ...            
https://almascience.org/datalink/sync?ID=uid://A002/X3216af/X31     applicati ...            
   https://almascience.org/datalink/sync?ID=uid://A001/X2df/Xe2     applicati ...            
   https://almascience.org/datalink/sync?ID=uid://A001/X2df/Xe2     applicati ...            
   https://almascience.org/datalink/sync?ID=uid://A001/X2df/Xe2     applicati ...            
                                              

### 1b. Select observations by time and ALMA band

You can further select rows of the table based on the data you are interested in. In the following example, we use datetime to select rows of the table corresponding to observations later than 2017.

In [13]:
from datetime import datetime

# Search table for observations that are from 2017 or later:
# ** Note that this will overwrite the original table.
q_results = q_results[[datetime.fromisoformat(i) > datetime(2018,1,1) for i in q_results['obs_release_date']]] 

# Create a new table by selecting by band,  e.g. specificall band 7 and 9:
# You can also modify this search to select by frequency or science goal.
q_results_b7_b9 = alma.query_object("M83",payload=dict(band_list = [7,9]))

# Search the new table for observations that are from 2017 or later:
q_results_b7_b9 = q_results_b7_b9[[datetime.fromisoformat(i) > datetime(2018,1,1) for i in q_results_b7_b9['obs_release_date']]]
print("M83, Band 7 and 9, newer than 2017:")
print(q_results_b7_b9)

M83, Band 7 and 9, newer than 2017:
                          access_url                          access_format ...       lastModified      collections
                                                                            ...                                    
------------------------------------------------------------- ------------- ... ----------------------- -----------
https://almascience.org/datalink/sync?ID=uid://A001/X888/X23e     applicati ... 2022-07-07T22:43:17.085            
https://almascience.org/datalink/sync?ID=uid://A001/X888/X23e     applicati ... 2022-07-07T22:43:17.085            
https://almascience.org/datalink/sync?ID=uid://A001/X888/X23e     applicati ... 2022-07-07T22:43:17.085            
https://almascience.org/datalink/sync?ID=uid://A001/X888/X23e     applicati ... 2022-07-07T22:43:17.085            
https://almascience.org/datalink/sync?ID=uid://A001/X888/X240     applicati ... 2022-07-07T22:43:17.085            
https://almascience.org/datalink/syn

In [3]:
#part 1: check if CO lines in band:

#get redshift
#from astroquery.ipac.ned import Ned
from astroquery.ipac.ned import Ned
z_result = Ned.query_object("M 83")#space is needed here!!
redshift = z_result['Redshift'][0]
print(redshift)


0.001711


In [4]:
#get CO lines from splatalogue
from astroquery.splatalogue import Splatalogue
l_results = Splatalogue.query_lines(1*u.GHz,1000*u.GHz,chemical_name = ' CO v = 0 ') 
print(l_results)
#a little cleanup by removing rows with masked values
l_results.remove_rows(np.where([c.data for c in l_results.mask.itercols()])[-1])              
#check if lines are in which projects 
#alma utils converts the frequency support into arrays holding the spectral windows' start and end frequencies
from astroquery.alma.utils import parse_frequency_support
#check which CO line is in which band
CO_loc = np.array([[any([a[0] < freq*u.GHz/(1+redshift) < a[1] for a in parse_frequency_support(q_row['frequency_support'])]) for q_row in q_results] for freq in l_results['Freq-GHz(rest frame,redshifted)']] ).T
CO_lin = ['CO '+' '.join(l_results['Resolved QNs'][a]) if any(a) else ' ' for a in CO_loc]
q_results['CO data']=CO_lin
print(q_results)

Species  Chemical Name  Freq-GHz(rest frame,redshifted) ... E_U (cm^-1)  E_U (K)  Linelist
------- --------------- ------------------------------- ... ----------- --------- --------
  COv=0 Carbon Monoxide                              -- ...     3.84503   5.53211      JPL
  COv=0 Carbon Monoxide                              -- ...     3.84503   5.53211     CDMS
  COv=0 Carbon Monoxide                      115.271202 ...         0.0       0.0    Lovas
  COv=0 Carbon Monoxide                      115.271202 ...     3.84503   5.53211    SLAIM
  COv=0 Carbon Monoxide                              -- ...    11.53492  16.59608      JPL
  COv=0 Carbon Monoxide                         230.538 ...         0.0       0.0    Lovas
  COv=0 Carbon Monoxide                         230.538 ...    11.53492  16.59608    SLAIM
    ...             ...                             ... ...         ...       ...      ...
  COv=0 Carbon Monoxide                              -- ...    80.73546 116.15965     CDMS

In [5]:
#eliminate data with no co lines
q_results = q_results[q_results['CO data'] != ' ']
#remove this column as it is masked and causes problems
q_results.remove_column('publication_year')


In [6]:
#download data and set a cache location
#only get fits files.
from astropy.table import unique

q_results = unique(q_results,'obs_id')
#remove the [0]!
uid_url_tab_list = [Alma.get_data_info(a, expand_tarfiles=True) for a in [q_results['obs_id'][0]]]
fits_urls = [[url for url in tab['access_url'] if '.fits' in url] for tab in uid_url_tab_list]
Alma.cache_location = '/big/external/drive/'
#files = [alma.download_files(url,cache=True) for url in fits_urls] tested and works
#print(files)


In [7]:

#Part 2: compare some catalogs, do cone search ~1deg
#cosmos field center:
from astropy.coordinates import SkyCoord
cos_cen = SkyCoord("10h00m24s+02d10m55s",frame = 'fk5')
c_results = Alma.query_region(cos_cen,radius = 0.5*u.deg)
print(c_results)

                           access_url                           access_format ... collections
                                                                              ...            
--------------------------------------------------------------- ------------- ... -----------
 https://almascience.org/datalink/sync?ID=uid://A001/X146a/X292     applicati ...            
 https://almascience.org/datalink/sync?ID=uid://A001/X146a/X292     applicati ...            
 https://almascience.org/datalink/sync?ID=uid://A001/X146a/X292     applicati ...            
 https://almascience.org/datalink/sync?ID=uid://A001/X146a/X26e     applicati ...            
 https://almascience.org/datalink/sync?ID=uid://A001/X146a/X26e     applicati ...            
 https://almascience.org/datalink/sync?ID=uid://A001/X146a/X26e     applicati ...            
 https://almascience.org/datalink/sync?ID=uid://A001/X146a/X296     applicati ...            
                                                            

In [8]:
c_results = unique(c_results,'obs_id')
print(c_results)

                           access_url                           access_format ... collections
                                                                              ...            
--------------------------------------------------------------- ------------- ... -----------
  https://almascience.org/datalink/sync?ID=uid://A001/X11a2/X11     applicati ...            
   https://almascience.org/datalink/sync?ID=uid://A001/X11d/X40     applicati ...       ari_l
   https://almascience.org/datalink/sync?ID=uid://A001/X11d/X44     applicati ...       ari_l
    https://almascience.org/datalink/sync?ID=uid://A001/X11d/X7     applicati ...       ari_l
    https://almascience.org/datalink/sync?ID=uid://A001/X11d/Xb     applicati ...       ari_l
    https://almascience.org/datalink/sync?ID=uid://A001/X11d/Xf     applicati ...       ari_l
  https://almascience.org/datalink/sync?ID=uid://A001/X121/X24f     applicati ...       ari_l
                                                            

In [9]:
from astroquery.esa.hsa import HSA
h_results = HSA.query_region(cos_cen, radius = 0.5*u.deg)
print(h_results)
print(len(h_results))
print(h_results.colnames)

ModuleNotFoundError: No module named 'astroquery.esa.hsa'

In [ ]:
c_catalog = SkyCoord(ra= c_results['s_ra'],dec=c_results['s_dec'])
h_catalog = SkyCoord(ra= h_results['ra'],dec=h_results['dec'])
idx,d2d,d3d = h_catalog.match_to_catalog_sky(c_catalog)
print(idx,d2d)

In [ ]:
print(c_results[idx])


In [ ]:
#use the query_tap mysql syntax to look for disk observations
#with a sensitivity better than 0.5 mjy/beam @ 10kms resolution
#at a frequency between 300 and 400 GHz
#
taptest = Alma.query_tap("select * from ivoa.ObsCore WHERE frequency > 300 AND frequency < 400 AND sensitivity_10kms < 5 AND science_keyword LIKE '%disk%'")
print(taptest)

In [ ]:
import astroquery
print(astroquery.__version__)